In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from ansys.mapdl.core import launch_mapdl
from ansys.mapdl import reader as pymapdl_reader

In [2]:
# Start an MAPDL instance
mapdl = launch_mapdl()

In [3]:
mapdl.finish()
mapdl.clear()
mapdl.prep7()

*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R2          24.2     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   00:55:59  NOV 24, 2024 CP=      0.750

                                                                               



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

## Kagome Lattice

In [88]:
design_width = 60/1000
design_height = 25/1000
total_width = 144/1000
total_z = 5/1000
# density = 0.5

wall_thickness = 0.4/1000
n_cell = 5
external_wall = False
external_wall_thickness = wall_thickness

In [89]:
cell_size = (design_height - wall_thickness)/n_cell/np.sqrt(3)

side_length = cell_size - 1/np.sqrt(3)*wall_thickness
design_part = mapdl.rectng(-1/1000, design_width, 0, design_height)

In [ ]:
for i in range(n_cell):
    j = 0
    while True:
        center = [cell_size/2 - np.sqrt(3)/2*wall_thickness + 2*j*cell_size - (i%2)*cell_size,
                  (cell_size + 1/np.sqrt(3)*wall_thickness)*np.sqrt(3)/2 + i*np.sqrt(3)*cell_size]
        # 조건문 넣을 필요 없이, design part 늘리면 해결되긴 함
        if center[0] - side_length < design_width:
            k_list = []
            for k in range(6):
                k_list.append(mapdl.k(x = center[0] + side_length*np.cos(np.pi/3*k), y = center[1] + side_length*np.sin(np.pi/3*k)))
            hole = mapdl.a(*k_list)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except:
                print(f"error2 at i={i}, j={j}")
        # 조건문 넣을 필요 없이, design part 늘리면 해결되긴 함
        if cell_size + 2*j*cell_size - (i%2)*cell_size < design_width:
            k1 = mapdl.k(x = cell_size + 2*j*cell_size - (i%2)*cell_size, y = wall_thickness + i*np.sqrt(3)*cell_size)
            k2 = mapdl.k(x = 2*cell_size - np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = wall_thickness + i*np.sqrt(3)*cell_size)
            k3 = mapdl.k(x = (3/2)*cell_size - 1/2*np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)/2*cell_size - 1/2*wall_thickness + i*np.sqrt(3)*cell_size)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except Exception as e:
                print(f"error2 at i={i}, j={j}")
                print(e)

            k1 = mapdl.k(x = cell_size + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)*cell_size + i*np.sqrt(3)*cell_size)
            k2 = mapdl.k(x = 2*cell_size - np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)*cell_size + i*np.sqrt(3)*cell_size)
            k3 = mapdl.k(x = (3/2)*cell_size - 1/2*np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)/2*cell_size + 3/2*wall_thickness + i*np.sqrt(3)*cell_size)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except Exception as e:
                print(f"error3 at i={i}, j={j}")
                print(e)

        if cell_size/2 - 1/np.sqrt(3)*wall_thickness + 2*j*cell_size > design_width:
            break

        j += 1

YES


In [92]:
mapdl.aplot()

Widget(value='<iframe src="http://localhost:4733/index.html?ui=P_0x1b48703f690_13&reconnect=auto" class="pyvis…

In [91]:
mapdl.rectng(x1=-total_width/2+design_width/2, x2=0, y1=0, y2=design_height)
mapdl.rectng(design_width, total_width/2+design_width/2, 0, design_height)
mapdl.aadd("all")

4

In [ ]:
mapdl.vext("ALL", dz = total_z)